In [1]:
import os
import sys
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
dirs = ["../.."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType, CustomTemplateType

model_type = CustomModelType.mixtral_moe_7b_instruct_gptq_int4
llm_engine = get_vllm_engine(
    model_type, 
    torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=1,
    # max_model_len=8192,
    # gpu_memory_utilization=1,
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 32,
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)



2024-03-30 17:11:37,511 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-03-30 17:11:37,514 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-03-30 17:11:37,553 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 ad87f4a589251d86333765b92ab59e41 and a total number of 972 components indexed
[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mixtral",
  "num_attention_heads": 32,
  "num_experts_per_tok": 2,
  "num_hidde

WARNING 03-30 17:11:38 config.py:618] Casting torch.bfloat16 to torch.float16.
WARNING 03-30 17:11:38 config.py:193] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 03-30 17:11:38 llm_engine.py:87] Initializing an LLM engine with config: model='/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4', tokenizer='/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=200, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=True, kv_cache_dtype=auto, device_config=cuda, seed=42)


OutOfMemoryError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacty of 23.65 GiB of which 10.06 MiB is free. Including non-PyTorch memory, this process has 23.63 GiB memory in use. Of the allocated memory 22.70 GiB is allocated by PyTorch, and 358.09 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [8]:
x = "CLAIM: Publication date: 2020-08-13, Content: Illinois Strengthens Face Mask Rules in Businesses\n\nTo accurately classify this claim, we need to analyze the relevant information provided. \n\nInformation 1: This is from 2013 and is not directly related to the claim about Illinois strengthening face mask rules in businesses in 2020. It provides general information about COVID-19 symptoms, risk factors, and prevention.\n\nInformation 2: This information confirms that Illinois lifted its indoor mask requirement on February 28, 2022, except where required by specific laws, rules, and regulations. It also mentions that the state was optimistic about this decision due to the decreasing trends of COVID-19 cases. The information does not directly contradict the claim about strengthening face mask rules in 2020, but it provides context about the situation in 2021-2022.\n\nInformation 3: This information reveals that on September 3, 2021, the Governor of Illinois signed an Executive Order requiring face coverings for individuals over the age of 2 in indoor public places. It also mandated vaccination for specific groups. This information supports the claim that Illinois strengthened face mask rules in businesses in 2020, as the Executive Order was issued in September 2021, which falls within the timeframe of the claim.\n\nInformation 4: This information provides guidance for businesses regarding the use of face coverings during the COVID-19 pandemic. It explains the definition of a face covering, exceptions for medical conditions or disabilities, and situations where masks are still required. This information does not directly contradict the claim but provides additional context about the implementation of face mask rules in businesses.\n\nInformation 5: This information is a list of statewide orders regarding face masks, but it is important to note that it was last updated on November 10, 2022, and the claim is about 2020. However, it does highlight the efforts of governors and public health officials to contain the spread of COVID-19 through stringent mitigation measures.\n\nIn conclusion, based on the available information, particularly Information 3, which confirms the implementation of stricter face mask rules in businesses in Illinois in September 2021, we can reasonably infer that the claim about Illinois strengthening face mask rules in businesses in 2020 is TRUE. However, it is essential to note that the specific action mentioned in the claim might not have occurred in 2020 but rather in 2021."

print(x)


CLAIM: Publication date: 2020-08-13, Content: Illinois Strengthens Face Mask Rules in Businesses

To accurately classify this claim, we need to analyze the relevant information provided. 

Information 1: This is from 2013 and is not directly related to the claim about Illinois strengthening face mask rules in businesses in 2020. It provides general information about COVID-19 symptoms, risk factors, and prevention.

Information 2: This information confirms that Illinois lifted its indoor mask requirement on February 28, 2022, except where required by specific laws, rules, and regulations. It also mentions that the state was optimistic about this decision due to the decreasing trends of COVID-19 cases. The information does not directly contradict the claim about strengthening face mask rules in 2020, but it provides context about the situation in 2021-2022.

Information 3: This information reveals that on September 3, 2021, the Governor of Illinois signed an Executive Order requiring fac

In [4]:
generation_config = VllmGenerationConfig(
    max_new_tokens=4096,
    temperature=0,
)

request_list = [
    {'query': 'Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.\
\n\nCLAIM:\nPublication date: 2020-08-13\nContent: Illinois Strengthens Face Mask Rules in Businesses\n\nINFORMATION:\nInformation 1:\nPublication date: 2013-08-06\nTitle: Illinois Strengthens Face Mask Rules in Businesses - WebMD\nContent:\nCOVID-19 is a new type of coronavirus that causes mild to severe cases. Here’s a quick guide on how to spot symptoms, risk factors, prevent spread of the disease, and find out what to do if you think you have it.\nInformation 2:\nPublication date: None\nTitle: press-release\nContent:\nIllinoisans can resume activities without wearing a mask indoors on February 28th except where required by federal, state, local, tribal, or territorial laws, rules, and regulations, including local business and workplace guidance. Federal requirements, in effect through at least March 18, include all transportation systems such as airports, planes, trains, and buses. "Preparing to repeal statewide masking mandates at the end of the month is aggressive and optimistic but reasonable," said Dr.\n"Broad mandates are not about individuals. They are put in place to help protect communities, businesses, and healthcare access. Repealing the mask mandate allows people to choose the mitigation layers that are best for them and I have no doubt that many should and will choose to keep mask rules."\nIf these trends continue — and we expect them to —then on Monday, February 28th, we will lift the indoor mask requirement for the State of Illinois," said Governor JB Pritzker. "I want to be clear: Many local jurisdictions, businesses and organizations have their own mask requirements and other mitigations that must be respected.\nSeth Trueger, a Northwestern emergency physician who is also immunocompromised. "Masking has helped slow the spread even in the face of omicron\'s transmissibility. We can and must use this time to further increase vaccination uptake & outreach, especially among children and other populations with low vaccination rates, so when the next wave comes, we will be even better prepared."\nInformation 3:\nPublication date: None\nTitle: Mask and Vaccine Requirements FAQ\'s\nContent:\nOn September 3, 2021, the Governor signed Executive Order 21-22 which requires all individuals over the age of 2 and who can medically tolerate a face covering to wear a face covering when in indoor public places. The Executive Order also requires health care workers, school personnel, higher education personnel and students, and employees and contractors of state-owned or operated congregate facilities to be fully vaccinated, as described in the Order\nInformation 4:\nPublication date: None\nTitle: FAQ for Businesses Concerning Use of Face-Coverings During COVID-19\nContent:\nA: Businesses reserve the right to refuse service to persons unable to comply with the requirement to wear a face covering, but they are required to provide a reasonable accommodation if it does not cause an undue hardship. Businesses are encouraged to inform customers there are exceptions to the requirement that all individuals must wear a mask.\nThese frequently asked questions are to provide guidance regarding the application of the face covering requirement in Executive Order 2021-10 for businesses and other places of public accommodation subject to Article 5 of the Illinois Human Rights Act, 775 ILCS 5/. A: A face covering is a mask or cloth face covering that covers your nose and mouth.\nExceptions may be made for individuals with medical conditions or disabilities that prevent them from safely wearing a face covering. For more information, refer to the questions on reasonable accommodations. A: Masks still must be worn by everyone on planes, buses, trains, and other forms of public transportation; in transportation hubs, such as airports and train and bus stations; in health care settings; and in congregate facilities, such as correctional facilities and homeless shelters.\nThese frequently asked questions are to provide guidance regarding the application of the face covering requirement in Executive Order 2021-10 for businesses and other places of public accommodation subject to Article 5 of the Illinois Human Rights Act, 775 ILCS 5/. When Face Coverings are Required Q: What does it mean to wear a face covering?\nInformation 5:\nPublication date: 2020-04-22\nTitle: Facing Your Face Mask Duties – A List of Statewide Orders | Littler ...\nContent:\n********************************************************NOTE: Given the reduction in activity on this topic,THIS POST WILL NO LONGER BE UPDATED, as of November 10, 2022.********************************************************Governors and public health officials across the country implemented stringent mitigation measures to help contain the spread of COVID-19.\n'},
    # {'query': "1+1=?"},
]


resp_list = inference_vllm(
    llm_engine, template, request_list, generation_config=generation_config, 
    # use_tqdm=True,
    # verbose=True, prompt_prefix="", output_prefix=""
)

for request, resp in zip(request_list, resp_list):
    print(f"query:\n{request['query']}")
    print()
    print(f"response:\n{resp['response']}")


query:
Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.

CLAIM:
Publication date: 2020-08-13
Content: Illinois Strengthens Face Mask Rules in Businesses

INFORMATION:
Information 1:
Publication date: 2013-08-06
Title: Illinois Strengthens Face Mask Rules in Businesses - WebMD
Content:
COVID-19 is a new type of coronavirus that causes mild to severe cases. Here’s a quick guide on how to spot symptoms, risk factors, prevent spread 